<a href="https://colab.research.google.com/github/rebeca07-pedrozo/Sheets-Automation/blob/main/Borrado_duplicados_en_emisiones_2_0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import pandas as pd
import numpy as np

In [75]:
emisiones = pd.read_csv('/content/Ventas3Agosto.csv')

In [ ]:
emisiones.dtypes

,0
clave_agente,int64
numero_poliza,int64
codigo_producto,int64
fecha_emision,object
nombre_producto,object
nombre_opcion_poliza,object
prima,float64
tipo_documento,object
NUMERO_DOCUMENTO,int64
NOMBRE,object


In [85]:
emisiones['prima'] = pd.to_numeric(emisiones['prima'], errors='coerce')


In [86]:
agrupado = emisiones.groupby('numero_poliza').agg({
    'clave_agente': 'first',
    'codigo_producto': 'first',
    'fecha_emision': 'first',
    'nombre_producto': 'first',
    'nombre_opcion_poliza': 'first',
    'prima': 'sum',
    'tipo_documento': 'first',
    'FECHA_PROCESO': 'first',
    # Los de abajo los vamos a procesar diferente
    'NUMERO_DOCUMENTO': lambda x: list(pd.unique(x)),
    'NOMBRE': lambda x: list(pd.unique(x)),
    'CORREO_1': lambda x: list(pd.unique(x)),
    'CELULAR_1': lambda x: list(pd.unique(x)),
}).reset_index()

In [87]:
def expandir_columnas(emisiones, columna_base, prefijo):
    max_items = emisiones[columna_base].apply(len).max()
    nuevas_col = pd.DataFrame(emisiones[columna_base].tolist(), columns=[f"{prefijo}_{i+1}" for i in range(max_items)])
    return pd.concat([emisiones.drop(columns=[columna_base]), nuevas_col], axis=1)

agrupado = expandir_columnas(agrupado, 'NUMERO_DOCUMENTO', 'NUMERO_DOCUMENTO')
agrupado = expandir_columnas(agrupado, 'NOMBRE', 'NOMBRE')
agrupado = expandir_columnas(agrupado, 'CORREO_1', 'CORREO')
agrupado = expandir_columnas(agrupado, 'CELULAR_1', 'CELULAR')

In [92]:
agrupado.dtypes


,0
numero_poliza,int64
clave_agente,int64
codigo_producto,int64
fecha_emision,object
nombre_producto,object
nombre_opcion_poliza,object
prima,float64
tipo_documento,object
FECHA_PROCESO,object
NUMERO_DOCUMENTO_1,int64


In [95]:
agrupado['prima'] = agrupado['prima'].fillna(0).astype('int64')
for col in agrupado.columns:
    if agrupado[col].dtype == 'float64':
        agrupado[col] = agrupado[col].fillna(0).astype('int64')

In [96]:
agrupado.to_csv("emisionesAgrupadas.csv", index=False)

